In [1]:
import numpy as np
import tensorflow as tf
import matplotlib.pylab as plt

import helpfuns as hf

Import Data

In [2]:
# Default parameters for data loading
DEFAULT_SHUFFLE = True

In [3]:
# Directory paths for data loading
tr_data_dir = '../../SC09_Preprocessed/'
tfrecords_fname = 'train.tfrecords'

In [ ]:
def _mapper(example_proto):
    features = {
        'signal' : tf.FixedLenSequenceFeature([1], tf.string, allow_missing = True)
        #'label': tf.FixedLenSequenceFeature([], tf.string, allow_missing = True)
    }
    example = tf.parse_single_example(example_proto, features)
    
    signal = tf.decode_raw(example['signal'], tf.float32)
    #label = tf.reduce_join(example['label'], 0)
        
    return signal#, label

    
dataset = tf.data.TFRecordDataset([tr_data_dir + tfrecords_fname])
dataset = dataset.map(_mapper)
dataset = dataset.shuffle(buffer_size = 1) #Batch_size = 1
dataset = dataset.apply(tf.contrib.data.batch_and_drop_remainder(1))
xs = dataset.make_one_shot_iterator().get_next()

sess = tf.Session()
x = sess.run(xs)
#print(x[0,0,0,5000:6000])
#dataset = dataset.shuffle(buffer_size=FLAGS.shuffle_buffer_size)
#dataset = dataset.map(map_func=parse_fn)
#dataset = dataset.batch(batch_size=FLAGS.batch_size)

Define Models

In [4]:
def PhaseShuffle(net, scope=None):
    """randomly perturbs the phase of each layer’s activations 
    by −2 to 2 samples before input to the next layer. 
    We use reflection padding to fill in the the missing samples"""
    rand_phase = tf.random_uniform([], 
                              minval=-2, 
                              maxval=2 + 1, 
                              dtype=tf.int32,
                              name="rand_sampling")
    pading_scheme = [[0, 0], [ tf.maximum(rand_phase, 0), tf.maximum(-rand_phase, 0)], [0, 0]]
    org_dims = net.get_shape().as_list()
    net = tf.pad(net,
                 paddings=pading_scheme,
                 mode='REFLECT',
                 name="refl_pading")[:, tf.maximum(-rand_phase, 0):tf.maximum(-rand_phase, 0)+org_dims[1]]
    net.set_shape(org_dims)
    net = tf.identity(net, name=scope+"_PhaseShuffle")
    return net

# Deconvolution Opperation
def conv1d_transpose(inputs,filters,kernel_width=25,stride=4,activation=tf.nn.relu,batchnorm = False,is_training=True,scope=None):

    net = tf.layers.conv2d_transpose(
        tf.expand_dims(inputs, axis=1),
        filters=filters,
        kernel_size=(1, kernel_width),
        strides=(1, stride),
        padding='SAME',
        activation=None,
        name=scope+"_conv1d")[:, 0]

    if batchnorm:
        net = tf.layers.batch_normalization(net, training=is_training, fused=True, name=scope+"_BN")
        net = activation(net, name=scope+"_act")
    else:
        net = activation(net, name=scope+"_actv")
    return net

def add_special_layers(net,scope=None, is_training=True, batchnorm=True, activation=None, shuffle=True):
    if shuffle:
        net = PhaseShuffle(net, scope=scope)
    if batchnorm:
        net = tf.layers.batch_normalization(net, training=is_training, fused=True, name=scope+"_BN")
    if activation != None:
        net = activation(net, name=scope+"_actv")
    return net

In [5]:
def generator(inputs, is_training):
    with tf.variable_scope('Generator'):
            net = tf.layers.dense(inputs,
                                  units=1024,
                                  kernel_initializer=None,
                                  name="fc1")
            net = tf.layers.batch_normalization(net, training=is_training, fused=True, name="fc1_BN")
            net = tf.nn.relu(net, name="fc1_ReLu")
            net = tf.reshape(net, [tf.shape(inputs)[0], 16, 1024], name="fc1_rs")
            net = conv1d_transpose(net,
                                   filters=512,
                                   kernel_width=25,
                                   stride=4, 
                                   batchnorm = True,
                                   is_training=is_training,
                                   scope="tc1")
            net = conv1d_transpose(net,
                                   filters=256,
                                   kernel_width=25,
                                   stride=4, 
                                   batchnorm = True,
                                   is_training=is_training,
                                   activation=tf.nn.relu,
                                   scope="tc2")
            net = conv1d_transpose(net,
                                   filters=128,
                                   kernel_width=25,
                                   stride=4, 
                                   batchnorm = True,
                                   activation=tf.nn.relu,
                                   is_training=is_training,
                                   scope="tc3")
            net = conv1d_transpose(net,
                                   filters=64,
                                   kernel_width=25,
                                   stride=4, 
                                   batchnorm = True,
                                   activation=tf.nn.relu,
                                   is_training=is_training,
                                   scope="tc4")
            net = conv1d_transpose(net,
                                   filters=1,
                                   kernel_width=25,
                                   stride=4, 
                                   batchnorm = True,
                                   activation=tf.nn.tanh,
                                   is_training=is_training,
                                   scope="tc5")  
            if is_training:
                update_ops = tf.get_collection(tf.GraphKeys.UPDATE_OPS)
            with tf.control_dependencies(update_ops):
                net = tf.identity(net, name="Generator_OUTPUT")
            
            return net

def discriminator(inputs, is_training):
    with tf.variable_scope('Discriminator'):
        net = tf.layers.conv1d(inputs,
                               filters=64, 
                               kernel_size=25, 
                               strides=4, 
                               padding='SAME',
                               kernel_initializer=None,
                               name="conv1")
        net = add_special_layers(net, 
                                 scope="conv1", 
                                 is_training=is_training, 
                                 batchnorm=True, 
                                 activation=tf.nn.leaky_relu, 
                                 shuffle=True)
        net = tf.layers.conv1d(inputs,
                               filters=128, 
                               kernel_size=25, 
                               strides=4, 
                               padding='SAME',
                               kernel_initializer=None,
                               name="conv2")
        net = add_special_layers(net, 
                                 scope="conv2", 
                                 is_training=is_training, 
                                 batchnorm=True, 
                                 activation=tf.nn.leaky_relu, 
                                 shuffle=True)
        net = tf.layers.conv1d(inputs,
                               filters=256, 
                               kernel_size=25, 
                               strides=4, 
                               padding='SAME',
                               kernel_initializer=None,
                               name="conv3")
        net = add_special_layers(net, 
                                 scope="conv3", 
                                 is_training=is_training, 
                                 batchnorm=True, 
                                 activation=tf.nn.leaky_relu, 
                                 shuffle=True)
        net = tf.layers.conv1d(inputs,
                               filters=512, 
                               kernel_size=25, 
                               strides=4, 
                               padding='SAME',
                               kernel_initializer=None,
                               name="conv4")
        net = add_special_layers(net, 
                                 scope="conv4", 
                                 is_training=is_training, 
                                 batchnorm=True, 
                                 activation=tf.nn.leaky_relu, 
                                 shuffle=True)
        net = tf.layers.conv1d(inputs,
                               filters=1024, 
                               kernel_size=25, 
                               strides=4, 
                               padding='SAME',
                               kernel_initializer=None,
                               name="conv5")
        net = add_special_layers(net, 
                                 scope="conv5", 
                                 is_training=is_training, 
                                 batchnorm=True, 
                                 activation=tf.nn.leaky_relu, 
                                 shuffle=False)
        net = tf.layers.flatten(net,name="flatten")
        net = tf.layers.dense(net, 
                              units=1,
                              kernel_initializer=None,
                              name="fc_out")[:, 0]
        net = tf.identity(net, name="Discriminator_OUTPUT")
        
        return net

Define Gan Graph

In [6]:
tf.reset_default_graph()
#Define problem specs
batch_size = 64
input_dim = 100

# Create Generator input
Gen_input = tf.random_uniform([batch_size, input_dim], minval=-1.0, maxval=1.0)
# Feed Generator
Generator_OUTPUT = generator(Gen_input, is_training=True)
# Connect Generator to Distcriminator
Discriminator_OUTPUT = discriminator(Generator_OUTPUT, is_training=True)


Instructions for updating:
`NHWC` for data_format is deprecated, use `NWC` instead


Run training Session

Run results